In [28]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import joblib
from tensorflow.keras.models import load_model


In [14]:
# Load and preprocess the data
with open("idmanual (1).json") as f:  # Replace "your_data.json" with your JSON file path
    data = json.load(f)

texts = []
labels = []

for entry in data:
    texts.append(entry['description'])
    labels.append(entry['class_id'])

texts = np.array(texts)
labels = np.array(labels)

In [15]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [16]:
# Pad sequences
max_seq_length = 100  # Replace with your desired sequence length
sequences = pad_sequences(sequences, maxlen=max_seq_length)

In [17]:
# Convert labels to integer format
label_to_id = {label: idx for idx, label in enumerate(np.unique(labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}
labels = np.array([label_to_id[label] for label in labels])

In [18]:
# Define early stopping callback
early_stopping = EarlyStopping(patience=2, restore_best_weights=True)

In [19]:
# Define the GRU model
embedding_dim = 100  # Replace with your desired embedding dimension
num_classes = len(np.unique(labels))
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_seq_length))
model.add(GRU(128))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Define early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Train the model
model.fit(sequences, labels, epochs=10, batch_size=32, verbose=1, callbacks=[early_stopping])

Epoch 1/10
1835/1835 [==============================] - 116s 63ms/step - loss: 1.6030 - accuracy: 0.5727
Epoch 2/10
1835/1835 [==============================] - 122s 66ms/step - loss: 0.4657 - accuracy: 0.8723
Epoch 3/10
1835/1835 [==============================] - 113s 61ms/step - loss: 0.2144 - accuracy: 0.9378
Epoch 4/10
1835/1835 [==============================] - 113s 61ms/step - loss: 0.1293 - accuracy: 0.9623
Epoch 5/10
1835/1835 [==============================] - 113s 61ms/step - loss: 0.0834 - accuracy: 0.9755
Epoch 6/10
1835/1835 [==============================] - 114s 62ms/step - loss: 0.0598 - accuracy: 0.9831
Epoch 7/10
1835/1835 [==============================] - 113s 62ms/step - loss: 0.0427 - accuracy: 0.9879
Epoch 8/10
1835/1835 [==============================] - 112s 61ms/step - loss: 0.0360 - accuracy: 0.9892
Epoch 9/10
1835/1835 [==============================] - 112s 61ms/step - loss: 0.0282 - accuracy: 0.9923
Epoch 10/10
1835/1835 [==============================] 

In [21]:
# Evaluate the model on the entire dataset
loss, accuracy = model.evaluate(sequences, labels)
print(f'Accuracy: {accuracy}')

1835/1835 [==============================] - 23s 13ms/step - loss: 0.0150 - accuracy: 0.9960
Accuracy: 0.9960308074951172


In [29]:
# Save the model
model.save('GRUtrinedmodel')
print("Model saved successfully.")


INFO:tensorflow:Assets written to: GRUtrinedmodel\assets


INFO:tensorflow:Assets written to: GRUtrinedmodel\assets


Model saved successfully.


In [30]:
# Load the saved model
loaded_model = load_model('GRUtrinedmodel')

In [ ]:
# Take user input and make predictions
user_input = input("Enter a description: ")
user_sequence = tokenizer.texts_to_sequences([user_input])
user_sequence = pad_sequences(user_sequence, maxlen=max_seq_length)
# Make predictions using the loaded model
prediction = loaded_model.predict(user_sequence)
predicted_class_id = np.argmax(prediction)
predicted_class_label = id_to_label[predicted_class_id]
print(f'Predicted Class: {predicted_class_label}')